# Starting 3Di Simulation Demo

## Install and Import required packages:

In [2]:
!pip install datetime
!pip install threedi_api_client
!pip install threedi_scenario_downloader

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [5]:
import arcpy

In [22]:
from datetime import datetime, timedelta
import datetime
from getpass import getpass
import pandas as pd
import json
from threedi_api_client.threedi_api_client import ThreediApiClient
from threedi_api_client.api import ThreediApi
from threedi_api_client.versions import V3Api
from threedi_api_client.openapi import ApiException
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import requests
from pathlib import Path
import numpy as np
import configparser
import os
import math
from time import sleep
from threedi_scenario_downloader import downloader as dl
from pathlib import Path

## Provide credentials and check connection 

In [7]:
API_HOST = "https://api.3di.live"
PERSONAL_API_KEY = "xNmWa0TI.fR6QPYdIzgKhQhc5iOOJVM7OW13lT2mC"


config = {
    "THREEDI_API_HOST": API_HOST,
    "THREEDI_API_PERSONAL_API_TOKEN": PERSONAL_API_KEY
}

api_client: V3Api = ThreediApi(config=config)

In [8]:
try:
    user = api_client.auth_profile_list()
except ApiException as e:
    print("Oops, something went wrong. Maybe you made a typo?")
else:
    print(f"Successfully logged in as {user.username}!")

Successfully logged in as louisdechamps!


## Find and select model

In [9]:
models = api_client.threedimodels_list(limit=5)  # limit to the first 5 results
for model in models.results:
    print(f"{model.name}")

Centrum_proj_2023 #4
assen_baggelhuizen - baggelhuizen_klimaatsom (1) #11
assen_baggelhuizen - baggelhuizen_klimaatsom (1) #10
assen_pittelo - pittelo_klimaatsom (1) #19
assen-kloosterveen - kloosterveen_klimaatsom (1) #13


In [10]:
my_model = api_client.threedimodels_list(name__icontains='assen_pittelo - pittelo_klimaatsom (1) #19')
print(my_model)

my_model = my_model.results[0]

{'count': 1,
 'next': None,
 'previous': None,
 'results': [{'breach_count': 1,
              'description': '',
              'disabled': False,
              'epsg': 28992,
              'extent_one_d': {'coordinates': [[6.531728641019742,
                                                52.994295524229415],
                                               [6.587804772187735,
                                                53.01697215747997]],
                               'type': 'LineString'},
              'extent_two_d': {'coordinates': [[6.530120259229756,
                                                52.99407988169934],
                                               [6.599757596521151,
                                                53.01828788980113]],
                               'type': 'LineString'},
              'extent_zero_d': {'coordinates': [[6.531832440627837,
                                                 52.99433452238827],
                                     

## Find organization ID to run the simulation under

In [11]:
organisations = api_client.organisations_list()
for organisation in organisations.results:
    print(f"{organisation.name}: {organisation.unique_id}")

Gemeente Assen: 04b793908f7c4b599c578c68203a2f0d


## Create Simulation and add event

In [12]:
simulation_templates = api_client.simulation_templates_list(simulation__threedimodel__id=my_model.id)

for simulation_template in simulation_templates.results:
    print(simulation_template.id, simulation_template.name)

assert simulation_templates.count > 0, f"No simulation templates found for threedimodel {my_model.name}"

simulation_template_id = simulation_templates.results[0].id

simulation_name = input('Name your simulation\n')

my_simulation = api_client.simulations_from_template(
    data={
        "template": simulation_template_id,
        "name": simulation_name,
        "organisation": '04b793908f7c4b599c578c68203a2f0d',
        "start_datetime": datetime(2020, 11, 15, 14, 0),
        "duration": 3600  # in seconds, so we simulate for 1 hour
    }
)

12463 Wadi WA Scholtenstraat 9.5 weg 10.0
Name your simulation
13012020231044
{'breach': [],
 'fileboundaryconditions': {'file': {'bucket': '3di',
                                     'etag': '6b1494cd3c6423b44621cf9a2cc337f9',
                                     'expiry_date': datetime.date(2123, 9, 30),
                                     'filename': 'boundaries_7be78286.json',
                                     'id': 821054,
                                     'meta': {'event_count': 1,
                                              'timeframes': [0, 599940]},
                                     'prefix': None,
                                     'related_object': 'https://api.3di.live/v3/simulations/157506/events/boundaryconditions/file/66703/',
                                     'size': 83,
                                     'state': 'processed',
                                     'state_description': 'File has been '
                                                   

 'wind': []}


## Add Events

In [ ]:
#Add rain timeseries event
rain = api_client.simulations_events_rain_timeseries_create(
    simulation_pk=my_simulation.id, data={
        'offset':0,
        'values': [[0, 0.005], [900, 0.002], [1800, 0.001], [2700, 0.0005], [3600, 0]],
        'units': 'm/s'}
)


events = api_client.simulations_events(my_simulation.id)
print(events)

## Start and check status

In [13]:
status = api_client.simulations_status_list(my_simulation.id)
print(status)

{'created': datetime.datetime(2023, 12, 13, 9, 44, 12, 587303, tzinfo=tzutc()),
 'exit_code': None,
 'id': 753500,
 'name': 'created',
 'paused': None,
 'time': 0.0}


In [14]:
api_client.simulations_actions_create(my_simulation.id, data={"name": "start"})

{'compute_cluster': None,
 'duration': None,
 'max_rate': None,
 'name': 'start',
 'simulation_runner': None,
 'timeout': 300}

In [15]:
while True:
    simulation_status = api_client.simulations_status_list(my_simulation.id)
    
    print(f"Simulation status: {simulation_status.name}")
    
    if simulation_status.name.lower() == 'finished':
        print("Simulation has finished.")
        break
    
    time.sleep(10)  # Wait for 10 seconds before the next iteration


Simulation status: starting
Simulation status: initialized
Simulation status: initialized
Simulation status: initialized
Simulation status: initialized
Simulation status: initialized
Simulation status: initialized
Simulation status: initialized
Simulation status: initialized
Simulation status: initialized
Simulation status: initialized
Simulation status: initialized
Simulation status: initialized
Simulation status: initialized
Simulation status: initialized
Simulation status: initialized
Simulation status: initialized
Simulation status: initialized
Simulation status: initialized
Simulation status: initialized
Simulation status: initialized
Simulation status: initialized
Simulation status: initialized
Simulation status: initialized
Simulation status: initialized
Simulation status: initialized
Simulation status: initialized
Simulation status: initialized
Simulation status: initialized
Simulation status: initialized
Simulation status: initialized
Simulation status: initialized
Simulation 

# Using Lizard to simulate and download

## Post Processing

In [16]:
scenario_name= input('Scenario name: ')
testdict = {
    "scenario_name": scenario_name,
    "process_basic_results": True,
}


api_client.simulations_results_post_processing_lizard_basic_create(simulation_pk=my_simulation.id, data = testdict)

api_client.simulations_results_post_processing_lizard_start_create(simulation_pk = my_simulation.id, data = testdict)
api_client.simulations_results_post_processing_lizard_status_list(simulation_pk=my_simulation.id)

Scenario name: EsriRainfall


{'created': datetime.datetime(2023, 12, 13, 9, 57, 38, 703452, tzinfo=tzutc()),
 'simulation': 'https://api.3di.live/v3/simulations/165737/',
 'status': 'requested'}

In [17]:
while True:
    scenario_status = api_client.simulations_results_post_processing_lizard_status_list(simulation_pk=my_simulation.id)
    
    print(f"Simulation status: {scenario_status.status}")
    
    if scenario_status.status.lower() == 'archived':
        print("Scenario is archived.")
        break
    
    time.sleep(10)  # Wait for 10 seconds before the next iteration

Simulation status: archiving
Simulation status: archiving
Simulation status: archiving
Simulation status: archiving
Simulation status: archiving
Simulation status: archiving
Simulation status: archiving
Simulation status: archiving
Simulation status: archived
Scenario is archived.


## Set Parameters

In [ ]:
#Provide login credentials
lizard_username = '__key__'
api_key = "lmMIYSJR.KtuoasnIzy0DA1BpZBSSHQsS96Xadg9U"

dl.set_api_key(api_key)

#Enter scenario information
srs = "EPSG:28992"
raster_resolution = 1
name = "EsriRainfall" #scenario_name

## Download Max Waterlevel

In [20]:
base = os.path.normpath(arcpy.env.workspace + os.sep + os.pardir)

base_dir = base
filename = r'water_lvl.tif'

out_path = os.path.join(base_dir, filename)
print(out_path)


scenarios = dl.find_scenarios_by_name(scenario_name, limit=1)
print(scenarios)
uuid = ' '
for d in scenarios:
    uuid = d['uuid']
    print(d['uuid'])

print(uuid)

dl.download_maximum_waterlevel_raster(uuid,srs,pathname = out_path)

C:\Users\dechamps\Documents\ArcGIS\Projects\3DiDemo\water_lvl.tif
[{'url': 'https://demo.lizard.net/api/v4/scenarios/3fb180be-fec7-426c-968b-7be7f2442e59/', 'uuid': '3fb180be-fec7-426c-968b-7be7f2442e59', 'name': 'EsriRainfall', 'organisation': {'url': 'https://demo.lizard.net/api/v4/organisations/04b79390-8f7c-4b59-9c57-8c68203a2f0d/', 'uuid': '04b79390-8f7c-4b59-9c57-8c68203a2f0d', 'name': 'Gemeente Assen'}, 'project': None, 'description': '', 'extra_metadata': {}, 'access_modifier': 'Private', 'created': '2023-12-13T09:57:40.704146Z', 'last_modified': '2023-12-13T09:57:42.897198Z', 'simulation_identifier': '165737', 'simulation_start': '2020-11-15T14:00:00Z', 'simulation_end': '2020-11-15T15:00:00Z', 'supplier': 'louisdechamps', 'source': '3Di', 'model_identifier': '60384', 'model_revision': '52415', 'model_name': 'assen_pittelo-pittelo_klimaatsom-1-19', 'has_raw_results': True, 'total_size': 224548210, 'spatial_bounds': {'west': 6.53012033373951, 'east': 6.599623455692816, 'north':

## Add to scene (Optional)

In [ ]:
aprx = arcpy.mp.ArcGISProject('current')
print(aprx.listMaps()[0].name)
my_map = aprx.listMaps()[0]
my_map.addDataFromPath(out_path)

## Multi-Raster Download

In [23]:

output_directory = os.path.normpath(arcpy.env.workspace + os.sep + os.pardir)
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

start_time = datetime.datetime(2000, 1, 1, 13, 0)
time_interval = datetime.timedelta(minutes=10)
end_time = datetime.datetime(2000, 1, 1, 16, 0)

while start_time < end_time:
    formatted_time_filename = start_time.strftime("%Y%m%d%H%M")  # For creating filename
    formatted_time_request = start_time.strftime("%Y-%m-%dT%H:%M")  # For request
    print(formatted_time_request)
    
    output_filename = f"water_depth_{formatted_time_filename}.tif"
    full_output_path = os.path.join(output_directory, output_filename)
    
    try:
        dl.download_waterdepth_raster(uuid, srs, 2, formatted_time_request, pathname=full_output_path)
        print(formatted_time_request + " printed")
    except Exception as e:
        print(f"Error downloading raster for {formatted_time_request}: {str(e)}")
    
    start_time += time_interval

2000-01-01T13:00
2000-01-01T13:00 printed
2000-01-01T13:10
2000-01-01T13:10 printed
2000-01-01T13:20
2000-01-01T13:20 printed
2000-01-01T13:30
2000-01-01T13:30 printed
2000-01-01T13:40
2000-01-01T13:40 printed
2000-01-01T13:50
2000-01-01T13:50 printed
2000-01-01T14:00
2000-01-01T14:00 printed
2000-01-01T14:10
2000-01-01T14:10 printed
2000-01-01T14:20
2000-01-01T14:20 printed
2000-01-01T14:30
2000-01-01T14:30 printed
2000-01-01T14:40
2000-01-01T14:40 printed
2000-01-01T14:50
2000-01-01T14:50 printed
